# A differential example from Hong et al 2020

Let us consider the following differential system with rational dynamics (see example 6.4 in [Hong et al](https://arxiv.org/abs/1801.08112)):

$$\left\{\begin{array}{ll}
    \dot x_1 & = a_1(x_2-x_1) - \frac{k_aV_mx_1}{k_ck_a + k_cx_3 + k_ax_1}\\
    \dot x_2 & = a_2(x_1-x_2)\\
    \dot x_3 & = b_1(x_4 - x_3) - \frac{k_cV_mx_3}{k_ck_a + k_cx_3 + k_ax_1}\\
    \dot x_4 & = b_2(x_3-x_4)
\end{array}\right.$$

This is a model arising from pharmacokinetics (see [Demignot et al](https://pubmed.ncbi.nlm.nih.gov/2855567/)) and the only output in the system is the fucntion $y = x_1$. Let us transform this problem so we can use `dalgebra` to solve the identifiability problem.

In [1]:
%display latex
from dalgebra import *

## we add some lines to do some time control on executions
import signal
from contextlib import contextmanager


@contextmanager
def timeout(time):
    # Register a function to raise a TimeoutError on the signal.
    signal.signal(signal.SIGALRM, raise_timeout)
    # Schedule the signal to be sent after ``time``.
    signal.alarm(time)

    try:
        yield
    except TimeoutError:
        print(f"Timeout of {time} seconds expired for execution")
    finally:
        # Unregister the signal so it won't be triggered
        # if the timeout is not reached.
        signal.signal(signal.SIGALRM, signal.SIG_IGN)
        
def raise_timeout(signum, frame):
    raise TimeoutError

The system has 4 state variables $x_1(t), x_2(t), x_3(t)$ and $x_4(t)$ and the differential system can be written using the parameters $k_a, k_c, V_m, a_1, a_2, b_1$ and $b_2$. We need to create these variables and set the differential ring such that everything is a constant:

In [2]:
R.<a_1,a_2,b_1,b_2,k_a,k_c,V_m> = QQ[]
DR = DifferentialRing(R)
## We update the variables
a_1,a_2,b_1,b_2,k_a,k_c,V_m = DR.gens()
## We check that all these are constants
print("All are constants -> ", all(el.derivative() == 0 for el in DR.gens()))

DR

All are constants ->  True


Differential Ring [Multivariate Polynomial Ring in a_1, a_2, b_1, b_2, k_a, k_c, V_m over Rational Field] with derivation [Map from callable <lambda>]

In order to build the system, we need to create the four differential variables $x_1(t), x_2(t), x_3(t)$ and $x_4(t)$. In the code, these will be represented by $\texttt{x1}$, $\texttt{x2}$, $\texttt{x3}$ and $\texttt{x4}$ such that $\texttt{xi[k]} = x_i^{(k)}(t)$. We do that with the class `DifferentialPolynomialRing`:

In [3]:
DPR.<x1,x2,x3,x4> = DifferentialPolynomialRing(DR)
DPR

Ring of differential polynomials in (x1, x2, x3, x4) over Differential Ring [Multivariate Polynomial Ring in a_1, a_2, b_1, b_2, k_a, k_c, V_m over Rational Field] with derivation [Map from callable <lambda>]

In [4]:
x1,x2,x3,x4

(x1_*, x2_*, x3_*, x4_*)

At this point we can create a differential system with the appropiate equations. Since $x_1(t)$ is our output variable that is the variable we do not need to eliminate, so the variables for teh system are the remaining $x_2(t), x_3(t)$ and $x_4(t)$:

In [5]:
denom = k_c*k_a + k_c*x3[0] + k_a*x1[0]
system = DifferentialSystem([
    x1[1]*denom - a_1*(x2[0]-x1[0])*denom + k_a*V_m*x1[0],
    x2[1] - a_2*(x1[0] - x2[0]),
    x3[1]*denom - b_1*(x4[0]-x3[0])*denom + k_c*V_m*x3[0],
    x4[1] - b_2*(x3[0]-x4[0])
], DPR, [x2,x3,x4])
system

System over [Ring of differential polynomials in (x1, x2, x3, x4) over Differential Ring [Multivariate Polynomial Ring in a_1, a_2, b_1, b_2, k_a, k_c, V_m over Rational Field] with derivation [Map from callable <lambda>]] with variables [(x2_*, x3_*, x4_*)]:
{
	k_a*x1_1*x1_0 + k_c*x1_1*x3_0 + a_1*k_a*x1_0^2 + (-a_1*k_a)*x1_0*x2_0 + a_1*k_c*x1_0*x3_0 + (-a_1*k_c)*x2_0*x3_0 + k_a*k_c*x1_1 + (a_1*k_a*k_c + k_a*V_m)*x1_0 + (-a_1*k_a*k_c)*x2_0 == 0
	(-a_2)*x1_0 + x2_1 + a_2*x2_0 == 0
	k_a*x1_0*x3_1 + b_1*k_a*x1_0*x3_0 + (-b_1*k_a)*x1_0*x4_0 + k_c*x3_1*x3_0 + b_1*k_c*x3_0^2 + (-b_1*k_c)*x3_0*x4_0 + k_a*k_c*x3_1 + (b_1*k_a*k_c + k_c*V_m)*x3_0 + (-b_1*k_a*k_c)*x4_0 == 0
	(-b_2)*x3_0 + x4_1 + b_2*x4_0 == 0
}

Let us try to execute blindly the differential resultant algorithm:

In [7]:
with timeout(3600):
    res = system.diff_resultant(alg_res="iterative", verbose=True)

Checking if there is any linear variable...
Forced iterative elimination. We do not look for linear variables
Several eliminations are needed --> we use recursion
Picking the best variable to start with...
Picked differential variable x2_*
Picked the pivot [k_a*x1_1*x1_0 + k_c*x1_1*x3_0 ...] for differential elimination
Computing the elimination for all pair of equations...
Checking if there is any linear variable...
Forced iterative elimination. We do not look for linear variables
Only one variable remains. We proceed to eliminate it in an algebraic fashion
Extending the system to eliminate x2_*...
Trying the extension (0, 0)
Trying the extension (1, 0)
Found the valid extension (1, 0)
Iterating to remove all the algebraic variables...
--------------------------------------------------
	Remaining variables: [x2_0, x2_1]
	Picking best algebraic variable to eliminate...
	Number of appearance for each variable: [3, 2]. Number of equations: 3
	Picked x2_1
	Picking the best 'pivot' to elim